In [ ]:
%matplotlib inline
%pylab inline

import menpo.io as mio
from menpo.feature import fast_dsift, dsift, sparse_hog
from menpo.landmark import labeller, ibug_face_66
from menpo.visualize import visualize_images

## Create data

In [ ]:
for image in mio.import_images('/vol/atlas/databases/lfpw/trainset/', 
                           verbose=True, max_images=1):
    image = image.crop_to_landmarks_proportion(0.5)
    image = image.rescale_landmarks_to_diagonal_range(200)
    labeller(image, 'PTS', ibug_face_66)

In [ ]:
visualize_images(image)

## Feature computation and patch extraction

In [ ]:
feature_callable = sparse_hog

n_iterations = 10

group = 'ibug_face_66'
patch_size = (17, 17)

Compute features over the whole image and extract patches from the resulting feature image:

In [ ]:
%%timeit 

feature_image = feature_callable(image)

for _ in range(n_iterations):
    feature_patches = feature_image.extract_patches_around_landmarks(
        group=group, patch_size=patch_size, as_single_array=True)

Extract patches drom the original image and compute features over each patch:

In [ ]:
%%timeit

feature_image = fast_dsift(image)

for _ in range(n_iterations):
    patches = image.extract_patches_around_landmarks(
        group=group, patch_size=patch_size, as_single_array=True)
    feature_patches = []
    for p in patches[:, 0, ...]:
        fp = feature_callable(p)
        feature_patches.append(fp)
    feature_patches = np.asarray(feature_patches)

## Response map computation

In [ ]:
from pyfftw.interfaces.numpy_fft import fft2 as fft2w, ifft2 as ifft2w
import numpy as np
from numpy.fft import ifft2, fft2


In [ ]:
#i = feature_callable(image).pixels
x = np.random.randn(68, 8, 16, 16)
f = np.random.randn(68, 8, 16, 16)

In [ ]:
%%timeit

r = np.real(ifft2(fft2(x) * f))

In [ ]:
%%timeit

r = np.real(ifft2w(fft2w(x) * f))

In [ ]:
i = np.random.randn(8, 128, 128).astype(np.float32)
f = np.random.randn(66, 8, 128, 128).astype(np.float32)

In [ ]:
%%timeit

ifft2(np.sum(fft2(i) * f, axis=1))

In [ ]:
%%timeit

for c in i:
    convolve2d(c, f[0][0], mode='same')